In [15]:
%matplotlib inline

import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn import preprocessing
from numpy.random import uniform
from scipy.spatial.distance import cdist
from sklearn.decomposition import PCA
import plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [2]:
df=pd.read_csv("C:/Users/38164/Master studije/Data Mining/04 DM Klasterovanje/Wholesale customers data.csv")
df.head()

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
0,2,3,12669,9656,7561,214,2674,1338
1,2,3,7057,9810,9568,1762,3293,1776
2,2,3,6353,8808,7684,2405,3516,7844
3,1,3,13265,1196,4221,6404,507,1788
4,2,3,22615,5410,7198,3915,1777,5185


## Preprocesiranje:

Informacije o podacima:

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440 entries, 0 to 439
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   Channel           440 non-null    int64
 1   Region            440 non-null    int64
 2   Fresh             440 non-null    int64
 3   Milk              440 non-null    int64
 4   Grocery           440 non-null    int64
 5   Frozen            440 non-null    int64
 6   Detergents_Paper  440 non-null    int64
 7   Delicassen        440 non-null    int64
dtypes: int64(8)
memory usage: 27.6 KB


Enkodiranje nominalnih atributa:

In [4]:
# Enkodiranje Channel atributa
ohe = OneHotEncoder(sparse=False)
ohe = ohe.fit(df[['Channel']])
prefix = 'Channel'
onehot_encoded = ohe.transform(df[['Channel']])
features_names_prefixed = [ f"{prefix}_{category}" for category in ohe.categories_[0]]
df = pd.concat([df, pd.DataFrame(onehot_encoded, columns=features_names_prefixed)], axis=1)
df.drop(['Channel'],axis=1, inplace=True)

In [5]:
# Enkodiranje Region atributa
ohe = OneHotEncoder(sparse=False)
ohe = ohe.fit(df[['Region']])
prefix = 'Region'
onehot_encoded = ohe.transform(df[['Region']])
features_names_prefixed = [ f"{prefix}_{category}" for category in ohe.categories_[0]]
df = pd.concat([df, pd.DataFrame(onehot_encoded, columns=features_names_prefixed)], axis=1)
df.drop(['Region'],axis=1, inplace=True)

In [6]:
df.head()

,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen,Channel_1,Channel_2,Region_1,Region_2,Region_3
0,12669,9656,7561,214,2674,1338,0.0,1.0,0.0,0.0,1.0
1,7057,9810,9568,1762,3293,1776,0.0,1.0,0.0,0.0,1.0
2,6353,8808,7684,2405,3516,7844,0.0,1.0,0.0,0.0,1.0
3,13265,1196,4221,6404,507,1788,1.0,0.0,0.0,0.0,1.0
4,22615,5410,7198,3915,1777,5185,0.0,1.0,0.0,0.0,1.0


Nedostajuce vrednosti i duplikati:

In [7]:
df.isnull().sum()

Fresh               0
Milk                0
Grocery             0
Frozen              0
Detergents_Paper    0
Delicassen          0
Channel_1           0
Channel_2           0
Region_1            0
Region_2            0
Region_3            0
dtype: int64

In [8]:
df.shape

(440, 11)

In [9]:
df.drop_duplicates().shape

(440, 11)

Deskriptivna analiza podataka

In [10]:
df.describe()

,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen,Channel_1,Channel_2,Region_1,Region_2,Region_3
count,440.000000,440.000000,440.000000,440.000000,440.000000,440.000000,440.000000,440.000000,440.0000,440.000000,440.000000
mean,12000.297727,5796.265909,7951.277273,3071.931818,2881.493182,1524.870455,0.677273,0.322727,0.1750,0.106818,0.718182
std,12647.328865,7380.377175,9503.162829,4854.673333,4767.854448,2820.105937,0.468052,0.468052,0.3804,0.309234,0.450397
min,3.000000,55.000000,3.000000,25.000000,3.000000,3.000000,0.000000,0.000000,0.0000,0.000000,0.000000
25%,3127.750000,1533.000000,2153.000000,742.250000,256.750000,408.250000,0.000000,0.000000,0.0000,0.000000,0.000000
50%,8504.000000,3627.000000,4755.500000,1526.000000,816.500000,965.500000,1.000000,0.000000,0.0000,0.000000,1.000000
75%,16933.750000,7190.250000,10655.750000,3554.250000,3922.000000,1820.250000,1.000000,1.000000,0.0000,0.000000,1.000000
max,112151.000000,73498.000000,92780.000000,60869.000000,40827.000000,47943.000000,1.000000,1.000000,1.0000,1.000000,1.000000


Normalizacija podataka:

In [11]:
minmax_scale = preprocessing.MinMaxScaler(feature_range=(0, 1))
scaled_feature = minmax_scale.fit_transform(df)
scaled_feature=pd.DataFrame(data=scaled_feature, columns=df.columns)
scaled_feature.head()

,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen,Channel_1,Channel_2,Region_1,Region_2,Region_3
0,0.112940,0.130727,0.081464,0.003106,0.065427,0.027847,0.0,1.0,0.0,0.0,1.0
1,0.062899,0.132824,0.103097,0.028548,0.080590,0.036984,0.0,1.0,0.0,0.0,1.0
2,0.056622,0.119181,0.082790,0.039116,0.086052,0.163559,0.0,1.0,0.0,0.0,1.0
3,0.118254,0.015536,0.045464,0.104842,0.012346,0.037234,1.0,0.0,0.0,0.0,1.0
4,0.201626,0.072914,0.077552,0.063934,0.043455,0.108093,0.0,1.0,0.0,0.0,1.0


## Implementacija K-means algoritma:

In [12]:
def euclidean(point, data):
    sum=0
    for i in range(len(point)):
        sum+=point[i]-data[i]
        sum=sum**2
    return np.sqrt(sum)  

In [16]:
def kmeans(data,n_clusters, no_of_iterations):
    idx = np.random.choice(len(data), n_clusters, replace=False)
    #Biranje centroida
    centroids = data[idx, :]   
    #nalazenje distance izmedju centroida i svih tacaka
    distances = cdist(data, centroids ,'euclidean') 
    #najblizi centroid
    points = np.array([np.argmin(i) for i in distances])
    for _ in range(no_of_iterations): 
        centroids = []
        for idx in range(n_clusters):
            #racuna se novi centroid kao srednja vredonst tacaka koje su mu dodeljene
            current_cent = data[points==idx].mean(axis=0) 
            centroids.append(current_cent)
        centroids = np.vstack(centroids)        
        distances = cdist(data, centroids ,'euclidean')
        points = np.array([np.argmin(i) for i in distances])        
    return points 

In [17]:
label = kmeans(scaled_feature.to_numpy(),6,1000)

In [19]:
scaled_feature['Cluster']=label
scaled_feature.head()

,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen,Channel_1,Channel_2,Region_1,Region_2,Region_3,Cluster
0,0.112940,0.130727,0.081464,0.003106,0.065427,0.027847,0.0,1.0,0.0,0.0,1.0,5
1,0.062899,0.132824,0.103097,0.028548,0.080590,0.036984,0.0,1.0,0.0,0.0,1.0,5
2,0.056622,0.119181,0.082790,0.039116,0.086052,0.163559,0.0,1.0,0.0,0.0,1.0,5
3,0.118254,0.015536,0.045464,0.104842,0.012346,0.037234,1.0,0.0,0.0,0.0,1.0,3
4,0.201626,0.072914,0.077552,0.063934,0.043455,0.108093,0.0,1.0,0.0,0.0,1.0,5


In [20]:
pca_1d = PCA(n_components=1)
pca_2d = PCA(n_components=2)
pca_3d = PCA(n_components=3)

PCs_1d = pd.DataFrame(pca_1d.fit_transform(scaled_feature))
PCs_2d = pd.DataFrame(pca_2d.fit_transform(scaled_feature))
PCs_3d = pd.DataFrame(pca_3d.fit_transform(scaled_feature))

PCs_1d.columns = ["PC1_1d"]
PCs_2d.columns = ["PC1_2d", "PC2_2d"]
PCs_3d.columns = ["PC1_3d", "PC2_3d", "PC3_3d"]

scaled_feature['Cluster']=label
plotX = pd.concat([scaled_feature,PCs_1d,PCs_2d,PCs_3d], axis=1, join='inner')
plotX["dummy"] = 0

cluster0 = plotX[plotX["Cluster"] == 0]
cluster1 = plotX[plotX["Cluster"] == 1]
cluster2 = plotX[plotX["Cluster"] == 2]
cluster3 = plotX[plotX["Cluster"] == 3]
cluster4 = plotX[plotX["Cluster"] == 4]
cluster5 = plotX[plotX["Cluster"] == 5]

init_notebook_mode(connected=True)

#Instructions for building the 3-D plot

#trace1 is for 'Cluster 0'
trace1 = go.Scatter3d(
                    x = cluster0["PC1_3d"],
                    y = cluster0["PC2_3d"],
                    z = cluster0["PC3_3d"],
                    mode = "markers",
                    name = "Cluster 0",
                    marker = dict(color = 'rgba(255, 128, 255, 0.8)'),
                    text = None)

#trace2 is for 'Cluster 1'
trace2 = go.Scatter3d(
                    x = cluster1["PC1_3d"],
                    y = cluster1["PC2_3d"],
                    z = cluster1["PC3_3d"],
                    mode = "markers",
                    name = "Cluster 1",
                    marker = dict(color = 'rgba(255, 128, 2, 0.8)'),
                    text = None)

#trace3 is for 'Cluster 2'
trace3 = go.Scatter3d(
                    x = cluster2["PC1_3d"],
                    y = cluster2["PC2_3d"],
                    z = cluster2["PC3_3d"],
                    mode = "markers",
                    name = "Cluster 2",
                    marker = dict(color = 'rgba(0, 255, 200, 0.8)'),
                    text = None)

#trace4 is for 'Cluster 3'
trace4 = go.Scatter3d(
                    x = cluster3["PC1_3d"],
                    y = cluster3["PC2_3d"],
                    z = cluster3["PC3_3d"],
                    mode = "markers",
                    name = "Cluster 3",
                    marker = dict(color = 'rgba(255, 255, 0, 0.8)'),
                    text = None)

#trace5 is for 'Cluster 4'
trace5 = go.Scatter3d(
                    x = cluster4["PC1_3d"],
                    y = cluster4["PC2_3d"],
                    z = cluster4["PC3_3d"],
                    mode = "markers",
                    name = "Cluster 4",
                    marker = dict(color = 'rgba(144, 238, 144, 0.8)'),
                    text = None)

#trace6 is for 'Cluster 5'
trace6 = go.Scatter3d(
                    x = cluster5["PC1_3d"],
                    y = cluster5["PC2_3d"],
                    z = cluster5["PC3_3d"],
                    mode = "markers",
                    name = "Cluster 5",
                    marker = dict(color = 'rgba(48, 25, 52, 0.8)'),
                    text = None)

data = [trace1, trace2, trace3, trace4, trace5, trace6]

title = "Visualizing Clusters in Three Dimensions Using PCA"

layout = dict(title = title,
              xaxis= dict(title= 'PC1',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'PC2',ticklen= 5,zeroline= False),
             )

fig = dict(data = data, layout = layout)

iplot(fig)
